<a href="https://colab.research.google.com/github/Arteric-Jeff-Knight/collabs/blob/master/DNSTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <= Run this code block once to load libraries:

!pip install dnspython validators

import dns.query
import dns.resolver
import dns.zone
import numpy as np
import pandas as pd                                     # Import pandas.


from IPython.display import display
import ipywidgets as widgets

l = widgets.Layout(flex='0 1 auto', height='60px', min_height='40px', width='auto')

import dns.query
import dns.resolver
import dns.zone
import numpy as np
import pandas as pd                                     # Import pandas.
import validators
import socket

dom_string = "arteric.com, www.arteric.com, fart"

ns_string = """ns-592.awsdns-10.net,garablebaldg.org
ns-201.awsdns-25.com
ns-1642.awsdns-13.co.uk
ns-1404.awsdns-47.org"""


class DnsTester:

    domains = []
    nameservers = {}
    results = None
    ip_errors = 0
    mx_errors = 0

    def __init__(self, domains: str, nameservers: str):

        self.resolver = dns.resolver.Resolver()
        self.new_resolver = dns.resolver.Resolver()

        # Validate the domains, exclude them from further processing if they are invalid
        self.domains = self.split_string(domains)
        for check in self.domains:
            if not validators.url('http://' + check):
                print(check, "not valid domain, removing from tests")
                self.domains.remove(check)

        # Validate the nameservers, exclude them from further processing if they are invalid
        input_nameservers = self.split_string(nameservers)
        for check in input_nameservers:
            if self.is_valid_ipv4_address(check) or self.is_valid_ipv6_address(check):
                self.nameservers[check] = check
            elif validators.url('http://' + check):
                try:
                    result = self.resolver.resolve(check, "A")
                    self.nameservers[check] = result[0].to_text()
                except Exception as e:
                    print(e,"Removing from nameservers")
            else:
                print(check, "not valid nameserver, removing from tests")

        data = []
        a_records = []
        new_a_records = []
        mx_records = []
        new_mx_records = []
        for domain in self.domains:
            try:
                result = self.resolver.resolve(domain, "A")
                for rdata in result:
                    a_records.append(rdata.to_text())
            except Exception as e:
                print(e)

            try:
                result = self.resolver.resolve(domain, "MX")
                for rdata in result:
                    mx_records.append(rdata.to_text().split(' ')[1])
            except Exception as e:
                print(e)

            for ns_name, ns_ip in self.nameservers.items():
                self.new_resolver.nameservers = [ns_ip]

                try:
                    result = self.new_resolver.resolve(domain, "A")
                    for rdata in result:
                        new_a_records.append(rdata.to_text())
                except Exception as e:
                    print(e)

                try:
                    result = self.new_resolver.resolve(domain, "MX")
                    for rdata in result:
                        new_mx_records.append(rdata.to_text().split(' ')[1])
                except Exception as e:
                    print(e)

                combined_a_records = self.unique_members_in_order(a_records, new_a_records)
                for record in combined_a_records:
                    data.append({
                        'domain': domain,
                        'ip': record if record in a_records else '',
                        'mx': '',
                        'new_ns': ns_name,
                        #'new_ns_ip': ns_ip,
                        'new_ip': record if record in new_a_records else '',
                        'new_mx': '',
                        'is_a': 1,
                        'is_mx' : 0
                    })

                combined_mx_records = self.unique_members_in_order(mx_records, new_mx_records)
                for record in combined_mx_records:
                    data.append({
                        'domain': domain,
                        'ip': '',
                        'mx': record if record in mx_records else '',
                        'new_ns': ns_name,
                        #'new_ns_ip': ns_ip,
                        'new_ip': '',
                        'new_mx': record if record in new_mx_records else '',
                        'is_a': 0,
                        'is_mx' : 1
                    })
        self.results = pd.DataFrame(data)
        
        self.results['same_ip'] = self.results['new_ip'] == self.results['ip']
        self.results['same_mx'] = self.results['new_mx'] == self.results['mx']

        self.ip_errors = len(self.results[self.results['same_ip'] == False])
        self.mx_errors = len(self.results[self.results['same_mx'] == False])



    @staticmethod
    def split_string(string):
        return string.replace(" ", "").replace("\n", ",").split(',')

    @staticmethod
    def default_configs() -> dict:
        return {

        }

    @staticmethod
    def is_valid_ipv4_address(address):
        try:
            socket.inet_pton(socket.AF_INET, address)
        except AttributeError:  # no inet_pton here, sorry
            try:
                socket.inet_aton(address)
            except socket.error:
                return False
            return address.count('.') == 3
        except socket.error:  # not a valid address
            return False

        return True

    @staticmethod
    def is_valid_ipv6_address(address):
        try:
            socket.inet_pton(socket.AF_INET6, address)
        except socket.error:  # not a valid address
            return False
        return True

    @staticmethod
    def unique_members_in_order(list_1: list, list_2: list) -> list:
        list_1_copy = list_1[:]
        list_1_copy.sort()
        set_1 = set(list_1_copy)
        list_2_copy = list_2[:]
        list_2.sort()
        set_2 = set(list_2_copy)
        list_2_items_not_in_list_1 = list(set_2 - set_1)
        combined = list(set(list_1_copy + list_2_items_not_in_list_1))
        combined.sort()
        return combined



domains = widgets.Textarea(
    value='arteric.com, www.arteric.com',
    placeholder='Comma delimited list of domains',
    description='Domain(s):',
    disabled=False,
    layout=l
)

namservers = widgets.Textarea(
    value="""ns-592.awsdns-10.net
ns-201.awsdns-25.com
ns-1642.awsdns-13.co.uk
ns-1404.awsdns-47.org""",
    placeholder='Comma delimited list of custom nameservers',
    description='Nameserver(s):',
    disabled=False,
    layout=l
)

display(domains)

display(namservers)


In [ ]:
#@title <= Enter data above and run this as many times as needed

dns_test = DnsTester(domains.value, namservers.value)

#dns_test.results.at[0, 'ip'] = '99.99.99.99'
#dns_test.results.at[0, 'same_ip'] = False
#dns_test.ip_errors  = 1

if dns_test.ip_errors:
  print(f"WARNING! Found {dns_test.ip_errors} A records out of {dns_test.results.is_a.sum()} that do not match!")
  display(dns_test.results[dns_test.results['same_ip'] == False])
else:
  print(f'All {dns_test.results.is_a.sum()} A records match.')
  
print()

if dns_test.mx_errors:
  print(f"WARNING! Found {dns_test.mx_errors} MX records out of {dns_test.results.is_mx.sum()} that do not match!")
  display(dns_test.results[dns_test.results['same_mx'] == False])
else:
  print(f'All {dns_test.results.is_mx.sum()} MX records match.')

